In [1]:
import csv
import functools
import itertools
import json
import re
from concurrent.futures import ThreadPoolExecutor
from time import perf_counter
from typing import List, Dict

import lxml
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm.contrib.concurrent import thread_map


c:\Users\Mourad\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
def get_price_range(min_price, max_price, session, properties_per_page=30):
    api_url = f"https://www.immoweb.be/en/search-results/house-and-apartment/for-sale?countries=BE&page=1&orderBy=newest&isALifeAnnuitySale=false&minPrice={min_price}&maxPrice={max_price}"
    num_pages = session.get(api_url).json()['marketingCount'] // properties_per_page + 1 # total nb of properties / nb of properties per page
    if num_pages > 333: # there is a hard limit of 333 pages
        mid_price = (min_price + max_price) // 2 
        return get_price_range(min_price, mid_price, session).union(get_price_range(mid_price, max_price, session)) # split the range in two. Union prevents duplicates.
    return get_ids_for_category(min_price, max_price, num_pages, session)

def get_ids_from_page(i, min_price, max_price, session):
    api_url = f"https://www.immoweb.be/en/search-results/house-and-apartment/for-sale?countries=BE&page={i}&orderBy=newest&isALifeAnnuitySale=false&minPrice={min_price}&maxPrice={max_price}"
    return set(result['id'] for result in session.get(api_url).json()['results'])

def get_ids_for_category(min_price, max_price, num_pages, session):
    return set(itertools.chain.from_iterable(thread_map(functools.partial(get_ids_from_page, min_price=min_price, max_price=max_price, session=session), range(1, num_pages+ 1), max_workers=64)))

def get_property(id, session): 
    property_url = f"http://www.immoweb.be/en/classified/{id}"

    try:
        req = session.get(property_url, timeout=10)
        tables = pd.read_html(req.text)
        df = pd.concat(tables).set_index(0).T
        df['id'] = id
        df = df.set_index('id')
        df_prop = df.loc[:, ~df.columns.duplicated()].copy()

        original_dict = {}

        window_data = re.findall("window.dataLayer =(.+?);\n", req.text, re.S)
        if window_data and json.loads(window_data[0]):
            list_of_property_info = json.loads(window_data[0])
            original_dict = list_of_property_info[0]['classified']

        for i in df_prop.index:
            original_dict[df_prop.loc[i, df_prop.columns[0]]] = df_prop.loc[i, df_prop.columns[1]]

        df_dict = pd.DataFrame([original_dict])
        df_dict = df_dict.set_index('id')
        
    except Exception as e:
        print(type(e))
        return pd.DataFrame()

    return df_dict


def get_properties_by_id(ids, session, max_workers=64):
    return pd.concat(thread_map(functools.partial(get_property, session=session), ids, max_workers=max_workers))

def get_properties():
    with requests.Session() as session:
        ids = get_price_range(min_price=0, max_price=10**6, session=session)
        return get_properties_by_id(ids, session)

In [14]:
import time

start = time.time()
properties = get_properties()
properties.to_csv('full_properties.csv')
end = time.time()

print(f"Time elapsed: {end - start} seconds.")
print(f"Number of properties: {len(properties)}")
print(f"Number of properties per second: {len(properties) / (end - start)}")
print(f"Number of seconds per property: {(end - start) / len(properties)}")

  4%|▍         | 2799/69695 [01:20<22:31, 49.50it/s]  

<class 'ValueError'>


  9%|▉         | 6251/69695 [03:07<23:37, 44.77it/s]  

<class 'ValueError'>


 22%|██▏       | 15587/69695 [07:49<2:16:45,  6.59it/s]

...

 95%|█████████▍| 66193/69695 [33:03<01:09, 50.50it/s]

<class 'ValueError'>


 96%|█████████▌| 66616/69695 [33:15<03:52, 13.22it/s]

<class 'ValueError'>
<class 'ValueError'>


100%|██████████| 69695/69695 [34:29<00:00, 33.68it/s]
